In [ ]:
"""

Apply the uncertainty of the parameters from Burgard et al. 2022 to show that 
uncertainty from parameters is lower than uncertainty from parameterisations

"""


In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import pandas as pd 
from tqdm.notebook import trange, tqdm
import time
import multimelt.melt_functions as meltf
import multimelt.plume_functions as pf
import summer_paper.useful_functions as uf
from multimelt.constants import *
import seaborn as sns
import os, glob
import distributed

In [ ]:
#home_path = '/Users/claraburgard/bettik_clara/'
home_path='/bettik/burgardc/'
#home_path='/data/users/burgardc/bettik_clara/'

In [ ]:
mod = 'UKESM1-0-LL'
scenario = 'historical'

to2300 = False

if scenario == 'historical':
    yystart = 1980 #1850
    yyend = 2014
else:
    if to2300:
        yystart = 2015
        yyend = 2300
    else:
        yystart = 2015
        yyend = 2100   

READ IN DATA

In [ ]:
inputpath_data=home_path+'/DATA/SUMMER_PAPER/interim/'
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
inputpath_profiles = home_path+'/DATA/SUMMER_PAPER/interim/T_S_PROF/CMIP/'+mod+'/'
inputpath_plumes = home_path+'/DATA/SUMMER_PAPER/interim/PLUMES/BedMachine_4km/'
inputpath_boxes = home_path+'/DATA/SUMMER_PAPER/interim/BOXES/BedMachine_4km/'

outputpath_simple_all_old = home_path+'/DATA/BASAL_MELT_PARAM/interim/SIMPLE/nemo_5km_06161821_oneFRIS/'
outputpath_simple_all_new = home_path+'/DATA/SUMMER_PAPER/interim/SIMPLE/nemo_5km_orig_christoph_oneFRIS/'
outputpath_melt = home_path+'/DATA/SUMMER_PAPER/processed/OCEAN_MELT_RATE_CMIP/'+mod+'/'


# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)
file_isf['isf_name'] = file_isf['isf_name'].astype(str)
file_isf['region'] = file_isf['region'].astype(str)

BedMachine_orig = xr.open_dataset(inputpath_data+'BedMachine_v2_aggregated4km_allvars.nc')
BedMachine_orig_cut = uf.cut_domain_stereo(BedMachine_orig, map_lim, map_lim)
file_draft = (BedMachine_orig_cut['thickness'] - BedMachine_orig_cut['surface']).where(file_isf['ISF_mask'] > 1)
file_isf_conc = BedMachine_orig_cut['isf_conc']

file_TS_list = []
for tt in tqdm(range(yystart,yyend+1)):
    file_T_orig = xr.open_dataset(inputpath_profiles+'T_mean_prof_50km_contshelf_'+mod+'_'+scenario+'_'+str(tt)+'.nc')
    file_S_orig = xr.open_dataset(inputpath_profiles+'S_mean_prof_50km_contshelf_'+mod+'_'+scenario+'_'+str(tt)+'.nc')
    file_TS_orig = xr.merge([file_T_orig.rename({'thetao':'theta_ocean'}), file_S_orig.rename({'so':'salinity_ocean'})]).sel(Nisf=rignot_isf).assign_coords({'time': tt})
    file_TS_list.append(file_TS_orig)
file_TS = xr.concat(file_TS_list, dim='time').rename({'z':'depth'})
file_TS['depth'] = -1*file_TS['depth']

depth_axis_old = file_TS.depth.values
depth_axis_new = np.concatenate((np.zeros(1),depth_axis_old))
file_TS_with_shallow = file_TS.interp({'depth': depth_axis_new})

grid_cell_area_file = xr.open_dataset(inputpath_data+'gridarea_ISMIP6_AIS_4000m_grid.nc').sel(x=file_isf.x,y=file_isf.y)
true_grid_cell_area = grid_cell_area_file['cell_area'].drop('lon').drop('lat')
cell_area_weight = true_grid_cell_area/(4000 * 4000)

lon = file_isf.longitude
lat = file_isf.latitude

xx = file_isf.x
yy = file_isf.y
dx = (xx[2] - xx[1]).values
dy = (yy[2] - yy[1]).values
grid_cell_area_const = abs(dx*dy)  
grid_cell_area_weighted = file_isf_conc * grid_cell_area_const * cell_area_weight

ice_draft_pos = file_draft
ice_draft_neg = -ice_draft_pos

isf_stack_mask = uf.create_stacked_mask(file_isf['ISF_mask'], file_isf.Nisf, ['y','x'], 'mask_coord')

box_charac_all_2D = xr.open_dataset(inputpath_boxes + 'BedMachine_4km_boxes_2D_oneFRIS.nc')
box_charac_all_1D = xr.open_dataset(inputpath_boxes + 'BedMachine_4km_boxes_1D_oneFRIS.nc')
plume_charac = xr.open_dataset(inputpath_plumes+'BedMachine_4km_plume_characteristics.nc')

param_var_of_int_2D = file_isf[['ISF_mask', 'latitude', 'longitude', 'dGL']]
param_var_of_int_1D = file_isf[['front_bot_depth_avg', 'front_bot_depth_max','isf_name']]

geometry_info_2D = plume_charac.merge(param_var_of_int_2D).merge(ice_draft_pos.rename('ice_draft_pos')).merge(grid_cell_area_weighted.rename('grid_cell_area_weighted')).merge(file_isf_conc.rename('isfdraft_conc'))
geometry_info_1D = param_var_of_int_1D

SIMPLE

In [ ]:
gammas_new_tuning = xr.open_dataset(outputpath_simple_all_new + 'gammas_simple_ALL.nc')
gammas_old_tuning = xr.open_dataset(outputpath_simple_all_old + 'gammas_simple_ALL.nc')
gammas_SMALL_tuning = xr.open_dataset(outputpath_simple_all_new + 'gammas_simple_SMALL.nc')
gammas_LARGE_tuning = xr.open_dataset(outputpath_simple_all_new + 'gammas_simple_LARGE.nc')

simple_param_types = ['linear_local', 'quadratic_local', 'quadratic_local_locslope',
                      'quadratic_mixed_mean', 'quadratic_mixed_locslope'] 

nisf_list_orig = geometry_info_1D.Nisf
T_S_profile = file_TS_with_shallow.ffill(dim='depth').bfill(dim='depth')



for mparam in simple_param_types:
    
    eval_1D_list = []
    for tuning_sort in ['LARGE','SMALL']: #'old'

        if tuning_sort == 'LARGE':
            nisf_list = nisf_list_orig.sel(Nisf=[10,11])
            gamma = gammas_LARGE_tuning['slope'].sel(param=mparam)
        elif tuning_sort == 'SMALL':
            nisf_list = nisf_list_orig.drop_sel(Nisf=[10,11])
            gamma = gammas_LARGE_tuning['slope'].sel(param=mparam)

        eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf(nisf_list, T_S_profile, 
                                                                geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                                mparam, gamma, U_param=True, 
                                                                box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, verbose=True)
        eval_1D_list.append(eval_1D)
        
    eval_1D_all = xr.concat(eval_1D_list,dim='Nisf')

    eval_1D_all.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_'+scenario+'_oneFRIS.nc')


In [ ]:
nisf_list_orig = geometry_info_1D.Nisf
T_S_profile = file_TS_with_shallow.ffill(dim='depth').bfill(dim='depth')


#########

mparam = 'lazero19'   


nisf_list = nisf_list_orig
gamma = 6.2e-4
E0 = 2.0e-2



eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf(nisf_list, T_S_profile, 
                                                 geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                 mparam, gamma, E0=E0, 
                                                 box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, verbose=True)

        

eval_1D.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_'+scenario+'_oneFRIS.nc')

#########

nisf_list_orig = geometry_info_1D.Nisf
T_S_profile = file_TS_with_shallow.ffill(dim='depth').bfill(dim='depth')
    
mparam = 'boxes_4_pismyes_picopno'
pism_version = 'yes'
nD_config = 4
        

nisf_list = nisf_list_orig
gamma = 0.87e-5
C = 2.8e6
        
eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf(nisf_list, T_S_profile, 
                                                         geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                         mparam, gamma, C=C, angle_option='local',
                                                         box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, box_tot=nD_config, 
                                                         box_tot_option='nD_config', pism_version=pism_version, picop_opt='no',
                                                         verbose=True)  

eval_1D.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_'+scenario+'_oneFRIS.nc')

RUN AS A LOOP OVER MODELS

In [ ]:
scenario = 'ssp585'


for mod in ['ACCESS-CM2','ACCESS-ESM1-5','CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
           'CanESM5','GFDL-CM4','GFDL-ESM4','GISS-E2-1-H','IPSL-CM6A-LR','MPI-ESM1-2-HR',
           'MRI-ESM2-0','UKESM1-0-LL']: #
    print(mod)
    
    if mod in ['CNRM-CM6-1','CNRM-ESM2-1']:
        to2300 = False
    elif mod in ['GISS-E2-1-H']:
        to2300 = True
    elif mod in ['ACCESS-CM2','ACCESS-ESM1-5','CESM2-WACCM','CanESM5','IPSL-CM6A-LR','MRI-ESM2-0']:
        to2300 = True
    elif mod in ['MPI-ESM1-2-HR','GFDL-CM4','GFDL-ESM4']:
        to2300 = False
    elif mod == 'UKESM1-0-LL':
        to2300 = True     
    elif mod == 'CESM2':
        to2300 = False        

    if scenario == 'historical':
        yystart = 1980 #1850
        yyend = 2014
    else:
        if to2300:
            yystart = 2015
            yyend = 2300
        else:
            yystart = 2015
            yyend = 2100   

    ### READ IN DATA
    inputpath_data=home_path+'/DATA/SUMMER_PAPER/interim/'
    inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
    inputpath_profiles = home_path+'/DATA/SUMMER_PAPER/interim/T_S_PROF/CMIP/'+mod+'/'
    inputpath_plumes = home_path+'/DATA/SUMMER_PAPER/interim/PLUMES/BedMachine_4km/'
    inputpath_boxes = home_path+'/DATA/SUMMER_PAPER/interim/BOXES/BedMachine_4km/'

    outputpath_simple_all_old = home_path+'/DATA/BASAL_MELT_PARAM/interim/SIMPLE/nemo_5km_06161821_oneFRIS/'
    outputpath_simple_all_new = home_path+'/DATA/SUMMER_PAPER/interim/SIMPLE/nemo_5km_orig_christoph_oneFRIS/'
    outputpath_melt = home_path+'/DATA/SUMMER_PAPER/processed/OCEAN_MELT_RATE_CMIP/'+mod+'/'


    # make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
    map_lim = [-3000000,3000000]

    file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
    nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
    rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)
    file_isf['isf_name'] = file_isf['isf_name'].astype(str)
    file_isf['region'] = file_isf['region'].astype(str)

    BedMachine_orig = xr.open_dataset(inputpath_data+'BedMachine_v2_aggregated4km_allvars.nc')
    BedMachine_orig_cut = uf.cut_domain_stereo(BedMachine_orig, map_lim, map_lim)
    file_draft = (BedMachine_orig_cut['thickness'] - BedMachine_orig_cut['surface']).where(file_isf['ISF_mask'] > 1)
    file_isf_conc = BedMachine_orig_cut['isf_conc']

    file_TS_list = []
    for tt in tqdm(range(yystart,yyend+1)):
        file_T_orig = xr.open_dataset(inputpath_profiles+'T_mean_prof_50km_contshelf_'+mod+'_'+scenario+'_'+str(tt)+'.nc')
        file_S_orig = xr.open_dataset(inputpath_profiles+'S_mean_prof_50km_contshelf_'+mod+'_'+scenario+'_'+str(tt)+'.nc')
        file_TS_orig = xr.merge([file_T_orig.rename({'thetao':'theta_ocean'}), file_S_orig.rename({'so':'salinity_ocean'})]).sel(Nisf=rignot_isf).assign_coords({'time': tt})
        file_TS_list.append(file_TS_orig)
    file_TS = xr.concat(file_TS_list, dim='time').rename({'z':'depth'})
    file_TS['depth'] = -1*file_TS['depth']
    
    depth_axis_old = file_TS.depth.values
    depth_axis_new = np.concatenate((np.zeros(1),depth_axis_old))
    file_TS_with_shallow = file_TS.interp({'depth': depth_axis_new})

    grid_cell_area_file = xr.open_dataset(inputpath_data+'gridarea_ISMIP6_AIS_4000m_grid.nc').sel(x=file_isf.x,y=file_isf.y)
    true_grid_cell_area = grid_cell_area_file['cell_area'].drop('lon').drop('lat')
    cell_area_weight = true_grid_cell_area/(4000 * 4000)

    lon = file_isf.longitude
    lat = file_isf.latitude

    xx = file_isf.x
    yy = file_isf.y
    dx = (xx[2] - xx[1]).values
    dy = (yy[2] - yy[1]).values
    grid_cell_area_const = abs(dx*dy)  
    grid_cell_area_weighted = file_isf_conc * grid_cell_area_const * cell_area_weight

    ice_draft_pos = file_draft
    ice_draft_neg = -ice_draft_pos

    isf_stack_mask = uf.create_stacked_mask(file_isf['ISF_mask'], file_isf.Nisf, ['y','x'], 'mask_coord')

    box_charac_all_2D = xr.open_dataset(inputpath_boxes + 'BedMachine_4km_boxes_2D_oneFRIS.nc')
    box_charac_all_1D = xr.open_dataset(inputpath_boxes + 'BedMachine_4km_boxes_1D_oneFRIS.nc')
    plume_charac = xr.open_dataset(inputpath_plumes+'BedMachine_4km_plume_characteristics.nc')

    param_var_of_int_2D = file_isf[['ISF_mask', 'latitude', 'longitude', 'dGL']]
    param_var_of_int_1D = file_isf[['front_bot_depth_avg', 'front_bot_depth_max','isf_name']]

    geometry_info_2D = plume_charac.merge(param_var_of_int_2D).merge(ice_draft_pos.rename('ice_draft_pos')).merge(grid_cell_area_weighted.rename('grid_cell_area_weighted')).merge(file_isf_conc.rename('isfdraft_conc'))
    geometry_info_1D = param_var_of_int_1D       

    ### RUN PARAMS
    nisf_list_orig = geometry_info_1D.Nisf
    T_S_profile = file_TS_with_shallow.ffill(dim='depth').bfill(dim='depth')

    gammas_new_tuning = xr.open_dataset(outputpath_simple_all_new + 'gammas_simple_ALL.nc')
    gammas_old_tuning = xr.open_dataset(outputpath_simple_all_old + 'gammas_simple_ALL.nc')
    gammas_SMALL_tuning = xr.open_dataset(outputpath_simple_all_new + 'gammas_simple_SMALL.nc')
    gammas_LARGE_tuning = xr.open_dataset(outputpath_simple_all_new + 'gammas_simple_LARGE.nc')

    simple_param_types = ['linear_local', 'quadratic_local', 'quadratic_local_locslope',
                          'quadratic_mixed_mean', 'quadratic_mixed_locslope'] 

    nisf_list_orig = geometry_info_1D.Nisf
    T_S_profile = file_TS_with_shallow.ffill(dim='depth').bfill(dim='depth')


    for mparam in simple_param_types:
        
        print(mparam)
        eval_1D_list = []
        for tuning_sort in ['LARGE','SMALL']: #'old'

            if tuning_sort == 'LARGE':
                nisf_list = nisf_list_orig.sel(Nisf=[10,11])
                gamma = gammas_LARGE_tuning['slope'].sel(param=mparam)
            elif tuning_sort == 'SMALL':
                nisf_list = nisf_list_orig.drop_sel(Nisf=[10,11])
                gamma = gammas_LARGE_tuning['slope'].sel(param=mparam)

            eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf(nisf_list, T_S_profile, 
                                                                    geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                                    mparam, gamma, U_param=True, 
                                                                    box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, verbose=False)
            eval_1D_list.append(eval_1D)

        eval_1D_all = xr.concat(eval_1D_list,dim='Nisf')

        eval_1D_all.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_'+scenario+'_oneFRIS.nc')


    #########

    mparam = 'lazero19'   
    print(mparam)

    nisf_list = nisf_list_orig
    gamma = 6.2e-4
    E0 = 2.0e-2



    eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf(nisf_list, T_S_profile, 
                                                     geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                     mparam, gamma, E0=E0, 
                                                     box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, verbose=False)



    eval_1D.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_'+scenario+'_oneFRIS.nc')

    #########

    nisf_list_orig = geometry_info_1D.Nisf
    T_S_profile = file_TS_with_shallow.ffill(dim='depth').bfill(dim='depth')

    mparam = 'boxes_4_pismyes_picopno'
    print(mparam)
    pism_version = 'yes'
    nD_config = 4


    nisf_list = nisf_list_orig
    gamma = 0.87e-5
    C = 2.8e6

    eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf(nisf_list, T_S_profile, 
                                                             geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                             mparam, gamma, C=C, angle_option='local',
                                                             box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, box_tot=nD_config, 
                                                             box_tot_option='nD_config', pism_version=pism_version, picop_opt='no',
                                                             verbose=False)  

    eval_1D.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_'+scenario+'_oneFRIS.nc')